In [1]:
import requests
from bs4 import BeautifulSoup
from random import choice

In [2]:
url = "https://www.sslproxies.org/"

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.content, 'html5lib')

In [5]:
list(map(lambda x:x[0]+':'+x[1], list(zip(map(lambda x:x.text, soup.findAll('td')[::8]), 
                                    map(lambda x:x.text, soup.findAll('td')[1::8])))))

['167.172.171.115:40061',
 '178.66.182.76:3128',
 '122.2.183.222:8080',
 '187.243.255.174:8080',
 '182.52.51.10:61124',
 '5.202.52.19:3128',
 '41.65.146.38:8080',
 '79.143.87.140:9090',
 '45.181.121.145:999',
 '1.186.40.2:54754',
 '209.45.111.198:45729',
 '187.243.253.2:8080',
 '103.162.205.100:8181',
 '131.108.118.27:8080',
 '185.189.211.70:8080',
 '101.109.255.18:50538',
 '138.99.164.90:3128',
 '202.62.67.209:53281',
 '103.73.74.217:2021',
 '78.138.131.248:3128',
 '185.131.62.250:53281',
 '182.16.171.42:43188',
 '95.85.24.83:8118',
 '179.108.123.210:51314',
 '185.67.95.179:3128',
 '1.20.102.102:38816',
 '95.165.187.202:45396',
 '176.62.188.158:56351',
 '103.11.106.69:8181',
 '112.78.170.28:8080',
 '103.6.104.105:38898',
 '157.230.95.4:8118',
 '201.59.201.92:39553',
 '213.108.5.226:3128',
 '103.143.196.98:8080',
 '51.222.67.213:32769',
 '160.238.227.153:30222',
 '181.143.106.162:52151',
 '65.18.114.254:55443',
 '119.2.47.46:8080',
 '194.8.85.146:3128',
 '149.56.47.132:443',
 '51.222.2

In [6]:
choice(list(map(lambda x:x[0]+':'+x[1], list(zip(map(lambda x:x.text, soup.findAll('td')[::8]), 
                                    map(lambda x:x.text, soup.findAll('td')[1::8]))))))

'112.78.170.28:8080'

In [7]:
def get_proxy():
    url = "https://www.sslproxies.org/"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')
    return {'https': choice(list(map(lambda x:x[0]+':'+x[1], list(zip(map(lambda x:x.text, soup.findAll('td')[::8]), 
                                    map(lambda x:x.text, soup.findAll('td')[1::8]))))))}

In [8]:
get_proxy()

{'https': '119.2.47.46:8080'}

In [9]:
def proxy_request(request_type, url, **kwargs):
    while 1:
        try:
            proxy = get_proxy()
            print("using proxy: {}".format(proxy))
            r = requests.request(request_type, url, proxies=proxy, timeout=5, **kwargs)
            break
        except:
            pass
    return r

In [10]:
r = proxy_request('get', 'https://youtube.com')

using proxy: {'https': '103.73.74.217:2021'}
using proxy: {'https': '52.60.43.64:80'}


In [11]:
r.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html>\n<head><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://youtube.com/</title></head>\n<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px;" onload="e=document.getElementById(\'captcha\');if(e){e.focus();}">\n<div style="max-width:400px;">\n<hr noshade size="1" style="color:#ccc; background-color:#ccc;"><br>\n<form id="captcha-form" action="index" method="post">\n<script src="https://www.google.com/recaptcha/api.js" async defer></script>\n<script>var submitCallback = function(response) {document.getElementById(\'captcha-form\').submit();};</script>\n<div id="recaptcha" class="g-recaptcha" data-sitekey="6LfwuyUTAAAAAOAmoS0fdqijC2PbbdH4kjq62Y1b" data-callback="submitCallback" data-s="55z-L9PKLXznYGW4kkyN-kAIOKxNmLDPa-z-Hbgyio-SsNbNg0n4c7s-O3IY4kmeYWoEpskVVtfjlxCt8BPYmOgZuYYq5tDUd8

In [ ]:
import socket
import threading
import select



SOCKS_VERSION = 5


class Proxy:
    def __init__(self):
        self.username = "username"
        self.password = "password"

    def handle_client(self, connection):
        # greeting header
        # read and unpack 2 bytes from a client
        version, nmethods = connection.recv(2)

        # get available methods [0, 1, 2]
        methods = self.get_available_methods(nmethods, connection)

        # accept only USERNAME/PASSWORD auth
        if 2 not in set(methods):
            # close connection
            connection.close()
            return

        # send welcome message
        connection.sendall(bytes([SOCKS_VERSION, 2]))

        if not self.verify_credentials(connection):
            return

        # request (version=5)
        version, cmd, _, address_type = connection.recv(4)

        if address_type == 1:  # IPv4
            address = socket.inet_ntoa(connection.recv(4))
        elif address_type == 3:  # Domain name
            domain_length = connection.recv(1)[0]
            address = connection.recv(domain_length)
            address = socket.gethostbyname(address)

        # convert bytes to unsigned short array
        port = int.from_bytes(connection.recv(2), 'big', signed=False)

        try:
            if cmd == 1:  # CONNECT
                remote = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                remote.connect((address, port))
                bind_address = remote.getsockname()
                print("* Connected to {} {}".format(address, port))
            else:
                connection.close()

            addr = int.from_bytes(socket.inet_aton(bind_address[0]), 'big', signed=False)
            port = bind_address[1]

            reply = b''.join([
                SOCKS_VERSION.to_bytes(1, 'big'),
                int(0).to_bytes(1, 'big'),
                int(0).to_bytes(1, 'big'),
                int(1).to_bytes(1, 'big'),
                addr.to_bytes(4, 'big'),
                port.to_bytes(2, 'big')
            ])
        except Exception as e:
            # return connection refused error
            reply = self.generate_failed_reply(address_type, 5)

        connection.sendall(reply)

        # establish data exchange
        if reply[1] == 0 and cmd == 1:
            self.exchange_loop(connection, remote)

        connection.close()

    
    def exchange_loop(self, client, remote):
        while True:
            # wait until client or remote is available for read
            r, w, e = select.select([client, remote], [], [])

            if client in r:
                data = client.recv(4096)
                if remote.send(data) <= 0:
                    break

            if remote in r:
                data = remote.recv(4096)
                if client.send(data) <= 0:
                    break

    
    def generate_failed_reply(self, address_type, error_number):
        return b''.join([
            SOCKS_VERSION.to_bytes(1, 'big'),
            error_number.to_bytes(1, 'big'),
            int(0).to_bytes(1, 'big'),
            address_type.to_bytes(1, 'big'),
            int(0).to_bytes(4, 'big'),
            int(0).to_bytes(4, 'big')
        ])


    def verify_credentials(self, connection):
        version = ord(connection.recv(1)) # should be 1

        username_len = ord(connection.recv(1))
        username = connection.recv(username_len).decode('utf-8')

        password_len = ord(connection.recv(1))
        password = connection.recv(password_len).decode('utf-8')

        if username == self.username and password == self.password:
            # success, status = 0
            response = bytes([version, 0])
            connection.sendall(response)
            return True

        # failure, status != 0
        response = bytes([version, 0xFF])
        connection.sendall(response)
        connection.close()
        return False


    def get_available_methods(self, nmethods, connection):
        methods = []
        for i in range(nmethods):
            methods.append(ord(connection.recv(1)))
        return methods

    def run(self, host, port):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind((host, port))
        s.listen()

        print("* Socks5 proxy server is running on {}:{}".format(host, port))

        while True:
            conn, addr = s.accept()
            print("* new connection from {}".format(addr))
            t = threading.Thread(target=self.handle_client, args=(conn,))
            t.start()


if __name__ == "__main__":
    proxy = Proxy()
    proxy.run("127.0.0.1", 3000)
    
#proxy.run("119.81.189.194:80", 3000)


* Socks5 proxy server is running on 127.0.0.1:3000
